# Transform Index Data

In [ ]:
import os
import pandas as pd
import psycopg2
from psycopg2 import sql

# database connection parameters
db_config = {
    'dbname': 'QF5214',
    'user': 'postgres',
    'password': 'qf5214',
    'host': '134.122.167.14',
    'port': '5555'
}

# CSV file path
csv_file_path = 'D:/01 NUS/QF5214 Data Engineering/TeamOne/index_data.csv'

# Extract schema and table name from CSV file name
schema_name = "datacollection"
table_name = os.path.splitext(os.path.basename(csv_file_path))[0]

# read CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# connect to the PostgreSQL database
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# ensure the schema exists
cursor.execute(sql.SQL("SET search_path TO {};").format(sql.Identifier(schema_name)))

cursor.execute(sql.SQL("""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_schema = %s AND table_name = %s
    );
"""), (schema_name, table_name))

table_exists = cursor.fetchone()[0]

if not table_exists:
    create_table_query = sql.SQL("CREATE TABLE {}.{} ({});").format(
        sql.Identifier(schema_name),
        sql.Identifier(table_name),
        sql.SQL(', ').join(
            sql.SQL("{} TEXT").format(sql.Identifier(col)) for col in df.columns
        )
    )
    cursor.execute(create_table_query)

# insert data into table
for _, row in df.iterrows():
    insert_query = sql.SQL("INSERT INTO {}.{} ({}) VALUES ({});").format(
        sql.Identifier(schema_name),
        sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, df.columns)),
        sql.SQL(', ').join(sql.Placeholder() for _ in df.columns)
    )
    cursor.execute(insert_query, tuple(row))

# commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print(f"data successfully written into {schema_name}.{table_name} ")


数据成功写入 datacollection.index_data 表


# Transform Stock Data

In [5]:
import os
import pandas as pd
import psycopg2
from psycopg2 import sql
from io import StringIO

# Database connection parameters
database = "QF5214"
username = "postgres"
password = "qf5214"
host = "134.122.167.14"
port = 5555

# CSV file path
csv_file_path = 'D:/01 NUS/QF5214 Data Engineering/TeamOne/stock_data.csv'

# Extract schema and table name from CSV file name
schema_name = "datacollection"
table_name = os.path.splitext(os.path.basename(csv_file_path))[0]

# Read CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname=database, 
    user=username, 
    password=password, 
    host=host, 
    port=port
)
cursor = conn.cursor()

# Ensure the schema exists
cursor.execute(sql.SQL("SET search_path TO {};").format(sql.Identifier(schema_name)))

# Check if the table exists
cursor.execute(sql.SQL("""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_schema = %s AND table_name = %s
    );
"""), (schema_name, table_name))

table_exists = cursor.fetchone()[0]

if not table_exists:
    # Create the table dynamically based on the DataFrame columns
    create_table_query = sql.SQL("CREATE TABLE {}.{} ({});").format(
        sql.Identifier(schema_name),
        sql.Identifier(table_name),
        sql.SQL(', ').join(
            sql.SQL("{} TEXT").format(sql.Identifier(col)) for col in df.columns
        )
    )
    cursor.execute(create_table_query)

# Use COPY to insert data in bulk
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, header=False)
csv_buffer.seek(0)

# Use COPY to insert the data from the StringIO buffer
cursor.copy_expert(
    sql.SQL("COPY {}.{} FROM STDIN WITH CSV").format(
        sql.Identifier(schema_name),
        sql.Identifier(table_name)
    ),
    csv_buffer
)

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print(f"data successfully written into {schema_name}.{table_name}")


data successfully written into datacollection.stock_data
